<a href="https://colab.research.google.com/github/nananatsu/blog/blob/master/vector_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 向量搜索学习笔记

 通过统计方法或机器学习，能够将现实世界的对象、概念转为向量表示， 再通过向量间的距离来判断它们的相似度。
 


向量有两种表示方法：[1.0,0.2,0.4,1.2,4.5,3.4]
- 稀疏向量，大部分元素0，少部分元素非0，如： [1.0,0,0,0,0,3.4]
  - 可以用三个分量来表示稀疏向量，减少存储所需空间
    -  向量大小
    -  非0元素的索引
    -  非0元素的值

In [ ]:
%pip install pyspark

In [ ]:
from pyspark.ml.linalg import Vector, DenseVector, SparseVector

In [ ]:
dv =DenseVector([1.0,0.2,0.4,1.2,4.5,3.4])
dv

DenseVector([1.0, 0.2, 0.4, 1.2, 4.5, 3.4])

In [ ]:
sv = SparseVector(6, {0:1.0, 5:3.4})
sv

SparseVector(6, {0: 1.0, 5: 3.4})

 常见相似度/距离计算方式：


In [ ]:
%pip install scipy

In [ ]:
from scipy.spatial import distance

 - 欧几里得距离（Euclidean Distance，欧氏距离），连接两点的直线距离，最常用的距离度量，随维度较增加适用性下降。
  
  $D(x,y) = (\mathop{∑}\limits_{i=1}^{n} (x_i - y_i)^2)^{1/2}$



In [ ]:
distance.euclidean([1, 0, 0], [0, 1, 0]) # 1.4142135623730951

1.4142135623730951

- 余弦相似度（Cosine Similarity），两个向量夹角的余弦，将向量归一化后等同于其内积，只考虑了向量的方向，没有考虑向量大小，向量间的差异没有充分考虑。

   $D(x,y) = cos(θ) = \frac{ x \cdot y} { \Vert x  \Vert  \Vert y  \Vert} = \frac{\mathop{∑}\limits_{i=1}^{n}  x_i × y_i} {\sqrt{\mathop{∑}\limits_{i=1}^{n}  x_i ^2}×\sqrt{\mathop{∑}\limits_{i=1}^{n}  y_i ^2}} $


  

In [ ]:
distance.cosine([1, 0, 0], [0, 1, 0])  # 1.0

1.0

- 曼哈顿距离（Manhattan Distance），计算实值向量间的距离，将平面网格化，向量只能水平或垂直移动得到的距离，在高维数据中不够直观，因为不是直线距离，给出的距离可能比欧几里得距离大。
  
   $ D(x,y) = \mathop{∑}\limits_{i=1}^{n}  \vert x_i - y_i  \vert $ 

 

In [ ]:
distance.cityblock([1, 0, 0], [0, 1, 0])  # 2

2

 - 切比雪夫距离（Chebyshev Distance），两个向量对应维的数据差值最大值，即沿着坐标轴计算的最大距离，适用场景特殊，一般不推荐使用。
  
   $D(x,y) = \mathop{max}\limits_{i} ( \vert x_i - y_i \vert )  = \mathop{lim} \limits_{k \to ∞ } ( \mathop{∑}\limits_{i=1}^{n}  \vert  x_i - y_i \vert ^k  ) ^ {1/k} $



In [ ]:
distance.chebyshev([1, 0, 0], [0, 1, 0]) # 1

1

  - 闵可夫斯基距离（Minkowski，闵氏距离），是欧几里得距离或曼哈顿距离的推广，$p$为1时为曼哈顿距离、$p$为2时为欧几里得距离、$p$为$∞$时为切比雪夫距离。
     
       $D(x,y) = (\mathop{∑}\limits_{i=1}^{n}  \vert x_i - y_i  \vert ^ p ) ^ {1/p} $
    
 

In [ ]:
distance.minkowski([1, 0, 0], [0, 1, 0], 1) # 2.0

2.0

In [ ]:
distance.minkowski([1, 0, 0], [0, 1, 0], 2) # 1.4142135623730951

1.4142135623730951

In [ ]:
distance.minkowski([1, 0, 0], [0, 1, 0], 3) # 1.2599210498948732

1.2599210498948732

 - 雅卡尔指数（Jaccard Index），样本交集大小除以并集大小，从1减去雅卡尔指数得到雅卡尔距离，受数据集大小影响很大。
  
  $D(x,y) = 1 - \frac{ \vert  x \bigcap y \vert }{ \vert  x \bigcup y \vert }  = 1 -  \frac{  x \cdot y }{ \Vert x \Vert ^2 + \Vert y \Vert ^2  -   x \cdot y }  = 1- \frac{ \mathop{∑}\limits_{i=1}^{n} x_i \times y_i }{   \mathop{∑}\limits_{i=1}^{n} x_i^2 + \mathop{∑}\limits_{i=1}^{n} y_i^2 -  \mathop{∑}\limits_{i=1}^{n} x_i \times y_i   }$ 



In [ ]:
distance.jaccard([1, 0, 0], [0, 1, 0]) # 1.0

1.0

  - 戴斯系数（Sørensen-Dice Index），与雅卡尔指数十分相似，可以视为两个集合的重叠率，相应差异函数1-戴斯系数没有三角形不等性质，不适合作为距离度量，戴斯系数在异构数据集中保持敏感，能够降低异常值的权重。
  
   $D(x,y) = \frac{ 2 \vert  x \bigcap y \vert }{ \vert  x   \vert +  \vert  y \vert }   =  \frac{ 2  \vert x \cdot y \vert }{ \Vert x \Vert ^2 + \Vert y \Vert ^2  }  =\frac{ 2 \mathop{∑}\limits_{i=1}^{n} x_i \times y_i }{   \mathop{∑}\limits_{i=1}^{n} x_i^2 + \mathop{∑}\limits_{i=1}^{n} y_i^2 } $



In [ ]:
distance.dice([1, 0, 0], [0, 1, 0]) #1.0

1.0

  - 汉明距离（Hamming Distance），两个向量间不同值的数量，当向量长度不等时很难使用并且不会考虑实际值，常用与网络传输中的纠错/检测。



In [ ]:
distance.hamming([1, 0, 0], [0, 1, 0]) # 0.6666666666666666

0.6666666666666666

  - 半正矢距离（Haversine Distance），给定经纬度球面上两点间的距离，实际上很少有这种情况，更多的是计算椭圆面上的距离（vincenty距离）。

  $D(xy) = 2r \: arcsin(  \sqrt{sin^2(\frac{𝛗_2 - 𝛗_1}{2}) + cos 𝛗_1⋅cos𝛗_2⋅sin^2(\frac{λ_2 - λ_1}{2} } ) ) $



In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

In [ ]:
haversine(1,0,0,1) # 157.24938127194397

157.24938127194397

  - 马哈拉诺比斯距离（Mahalanobis Distance，马氏距离），用于计算点与分布间的距离。给定分布D，均值$μ = (μ_1,μ_2,...,μ_p)$，协方差矩阵S，则点$x = (x_1.x_2,...,x_n)$与分布D的距离为：
     
      $D(x,D) = ( (x-μ ) ^T S ^ {-1} (x-μ) )^{1/2}$
    
    两个服从分布D的随机掉x、y间的马氏距离为：
      
      $D(x,y;D) = ( (x-y ) ^T S ^ {-1} (x-y) )^{1/2}$
    
     当协方差矩阵是单位矩阵时，可以将其等效为欧氏距离（$ σ_i$为$x_i$的标准差）：
        
       $D(x,y) = ( \frac{ \mathop{∑}\limits_{i=1}^{n} (x_i - y_i)^2}{ σ_i^2 })^{1/2}$


 

In [ ]:
iv = [[1, 0.5, 0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]]

In [ ]:
distance.mahalanobis([1, 0, 0], [0, 1, 0], iv) # 1.0

1.0

- 堪培拉距离（Canberra Distance），曼哈顿距离的加权版本。
    
  $D(x,y) = \mathop{∑}\limits_{i=1}^{n}  \frac{ \vert x_i - y_i  \vert}{ \vert x_i  \vert +  \vert y_i  \vert} $


 

In [ ]:
distance.canberra([1, 0, 0], [0, 1, 0]) # 2.0

2.0

 - 布雷-柯蒂斯相异度（Bray-curtis dissimilarity）：
   
   $D(x,y) = \frac{ \mathop{∑}\limits_{i=1}^{n}  \vert x_i - y_i  \vert }{ \mathop{∑}\limits_{i=1}^{n}  \vert x_i + y_i  \vert}$


 

In [ ]:
distance.braycurtis([1, 0, 0], [0, 1, 0]) # 1.0

1.0

 - KL散度（Kullback–Leibler divergence），衡量两个分布的相似性，离散概率分布$P$、$Q$在同一样本空间$\chi$，$Q$到$P$的KL散度为：
  
   $D_{KL}( P || Q ) =  \mathop{∑}\limits_{x \in \chi} P(x) log (\frac{P(x)} {Q(x)})$

   当$P$、$Q$是绝对连续的，$Q$到$P$的KL散度可定义为：
    
     $D_{KL}( P || Q ) = \int_{x} log( \frac{P(dx)} {Q(dx) }) P(dx) = \int_{x}  \frac{P(dx)}{Q(dx)} log( \frac{P(dx)} {Q(dx) }) Q(dx) $
      
    $μ$是$\chi$上的任意测度，对于概率密度$p$存在 $P(dx) = p(x)μ(dx)$ 、$Q(dx) = q(x)μ(dx)$，$Q$到$P$的KL散度可定义为：

      $D_{KL}( P || Q ) = \int_{x} p(x) log( \frac{p(x)} {q(x) }) μ(dx) $


In [ ]:
import scipy
import numpy as np

vec = scipy.special.rel_entr([1, 0, 0], [0, 1, 0])    
kl_div = np.sum(vec)

kl_div

inf

- JS散度（Jensen–Shannon divergence），是KL散度的平滑版本。

  $D_{JS}(P || Q) = \frac{1}{2}D_{KL}(P || M)  + \frac{1}{2}D_{KL}(Q || M)$，其中$ M =\frac{1}{2}(P+Q) $

 

In [ ]:
distance.jensenshannon([1.0, 0.0, 0.0], [0.0, 1.0, 0.0], 2.0) # 1.0

1.0

相似性搜索（向量搜索），给定一组向量和查询向量，从向量集中找到最相似的项目。

 KNN（k-nearest neighbors algorithm，k-最近邻居算法），在向量空间中为给定查询向量找到最近的向量，k-NN在查询时需要查询向量与向量集中每个向量的距离。

 ANN（approximately nearest neighbors，近似最近邻居），为减少KNN这类算法的计算复杂度，通过建立索引结构来缩小搜索空间以缩短查询时间。

Faiss（Facebook AI Similarity Search）是一个流行的相似性搜索库，我么可以将向量存储到Faiss中进行索引，再用查询向量从Faiss中找到最相似的向量。

BERT是一种流行的transform模型能将大量信息编码为一组密集向量，编码后通常会有512个密集向量，每个密集向量包含768个值。

Sentence-BERT对BERT进行修改允许创建能够表示完整序列的单个向量。

安装相关库
- sentence-transformers 将语句转为向量
- faiss 用于计算两个向量的相似度

In [ ]:
%pip install pandas sentence-transformers torch faiss-gpu

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
from google.colab import drive

挂载google drive到/content/drive/

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


从google drive读取数据文件去重，使用Sentence-BERT模型将语句转为密集向量。

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/train_data/SICK_train.txt', sep='\t')

sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them

# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)

print(sentence_embeddings.shape)

(4802, 768)


以向量维度初始化 faiss索引，再将编码后向量加入索引

- IndexFlatL2用了测量查询向量与加载到索引向量间的欧几里得距离

In [ ]:
d = sentence_embeddings.shape[1]

index = faiss.IndexFlatL2(d)
index.add(sentence_embeddings)

将一个查询语句编码向量，在索引中查询4个最相似向量

In [ ]:
k = 4
xq = model.encode(["Someone sprints with a football"])


In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4577 1202 1117 4633]]
CPU times: user 6.58 ms, sys: 1.01 ms, total: 7.59 ms
Wall time: 7.66 ms


每次查询时会与将查询向量与索引上所有向量进行比较，大量数据时索引会变得越来越慢。

一种流行的优化方法是划分为Voronoi单元，索引的每个成员向量都在一个Voronoi单元中，在进行新的查询时，先测量查询向量与Voronoi单元基点的距离，找到最近的Voronoi单元将搜索范围限定在该Voronoi单元。这样缩小搜索范围，生成了一个近似答案，而非精确答案。
- Voronoi图是一种空间分割算法，将平面分割为靠近一组给定目标的区域，最简单的情况下这些目标是平面上有限的点（称为种子、基点、生成器），每个基点对应一个区域称为Voronoi单元，单元中所有点距离该基点最近（距离其他基点更远）。

使用IndexIVFFlat添加了聚类，在添加索引数据前需要对数据进行训练。

将IndexFlatL2作为量化器得到IndexIVFFlat总的分区索引。

In [ ]:
nlist = 50  # Voronoi单元数量

quantizer = faiss.IndexFlatL2(d)

index = faiss.IndexIVFFlat(quantizer, d, nlist)

index.train(sentence_embeddings)

index.add(sentence_embeddings)

再次进行查询，得到结果与之前一致。

如未返回最优结果，可以通过增大index.nprobe（定义搜索多少个邻近的Voronoi单元）来增大搜索范围，相应的会降低搜索速度。

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4577 1202 1117 4633]]
CPU times: user 1.77 ms, sys: 0 ns, total: 1.77 ms
Wall time: 1.43 ms


In [ ]:
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[4577 1202 1117 4633]]
CPU times: user 1.76 ms, sys: 0 ns, total: 1.76 ms
Wall time: 1.77 ms


如果存储的向量都是完整的，数据集很大时，这将成为问题。

Faiss中使用Product Quantization (PQ，乘积量化)来压缩向量，PQ可以视为一个额外的近似步骤，IVF通过缩小搜索范围进行近似，PQ通过距离/相似性计算来进行近似。
- 将原始向量拆分为几个子向量；
- 对每个子向量执行聚类，为每个子向量集创建质心；
- 用距离子向量最近的质心id替换子向量；

In [ ]:
m = 8  #最终压缩向量中质心数量
bits = 8 # 质心的位数

quantizer = faiss.IndexFlatL2(d)  # 继续使用L2距离进行量化
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

index.train(sentence_embeddings)

index.add(sentence_embeddings)

In [ ]:
index.nprobe = 10

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[1202 4577 4633 1970]]
CPU times: user 1.65 ms, sys: 0 ns, total: 1.65 ms
Wall time: 1.67 ms


参考：
- 数据科学中的九种距离度量 <https://towardsdatascience.com/9-distance-measures-in-data-science-918109d069fa>